In [30]:
%matplotlib widget
%run '../_libtoolbox/init-notebook.py'
%run '../_libtoolbox/tbwidgets.py'
%run '../_libtoolbox/header.py'

&nbsp;<a href="https://nuflo.de"><img hspace="45%" align="middle" src="../_libtoolbox/nuflo-logo-N-512x512.png" alt="logo-N" width="8%" /></a>

<center><h1>nuflo Engineering Toolbox</h1></center>

[nuflo is an engineering company](https://nuflo.de) that supports digitalization of engineering processes in fluid mechanics.

*The code for this tool can be found over at [github](https://github.com/JohK/engineering-toolbox).*  
*More interactive tools are located at [toolbox.nuflo.de](http://toolbox.nuflo.de).*


Boundary Layer Thickness on a Flat Plate
---
This is a tool for a quick estimation of the boundary layer thickness on a flat plate. For this estimation we use the momentum loss thickness based on two simple correlations:

The momentum loss thickness of a laminar boundary layer $\delta_l$ (Blasius solution):
$$ \delta_l = 0.664 \cdot \sqrt{\frac{\nu \cdot x}{u}} $$

For a turbulent flow:
$$ \delta_t = 0.0142 \cdot x \cdot \left(\frac{u\cdot x}{\nu}\right)^\left(-\frac{1}{7}\right) $$

Depending on the physical problem these may not be the right choice, if unsure ask an experienced engineer.

In [2]:
@np.vectorize
def momentum_boundarylayer(x, u, gas):
    rho = gas.density
    nu = gas.kinematic_viscosity
    
    fdelta_laminar = lambda x: 0.664*np.sqrt(nu*x/u)
    fdelta_turbulent = lambda x: 0.0142*(u*x/nu)**(-1/7)*x
    
    try:
        boundarylayer_thickness = [np.max(fdelta_laminar(xi), fdelta_turbulent(xi)) for xi in x]
    except TypeError:
        boundarylayer_thickness = np.max([fdelta_laminar(x), fdelta_turbulent(x)])
    
    # find where laminar and turbulent boundary layer thickness intersect
    sol = root_scalar(lambda x: fdelta_laminar(x)-fdelta_turbulent(x), bracket=[1e-6,10e6])
    if sol.converged:
        transition_position = sol.root
    else:
        transition_position = np.nan
    
    return(boundarylayer_thickness,
           transition_position)

In [29]:
def plot_init():
    fig = plt.figure(figsize=(4, 4), dpi=100, constrained_layout=False)
    fig.canvas.header_visible = False
    fig.canvas.toolbar_visible = True
    fig.canvas.resizable = True
    fig.canvas.layout.min_height = '300px'
    fig.canvas.layout.min_width = '400px'
    fig.canvas.layout.flex='1 1 auto'
    fig.canvas.layout.width = 'auto'
    
    blline, = plt.plot(0, 0, '-', label='boundary layer thickness')
    trline, = plt.plot(0, 0, '.', label='laminar-turbulent transition')
    warningtext = plt.text(0.2, 0.5,
                            'Data not available, or\nnot a fluid in the property range',
                            transform=plt.gca().transAxes, 
                            bbox=dict(facecolor='red', alpha=0.2))
    plt.legend()
    plt.xlabel('length of flat plate in m')
    plt.ylabel('boundary layer thickness in mm')
    
    plt.grid()
    #plt.show()

#    fig.canvas.layout.display = 'flex'
    #fig.canvas.layout.height = '600px'
    
    def update(platelength, u, pressure, temperature, gasname):
        g = gas.state(gasname, 273.15+temperature, pressure)
        desc = '{:s}: T={:.1f}°C, p={:.3f}bar'.format(gasname, temperature, pressure/1e5)
        x = np.linspace(0.0000001, platelength, num=1000)
        fig.gca().set_title(desc)
        if not np.isnan(g.density):
            bl, transition = momentum_boundarylayer(x, u, g)
            transition_bl, _ =  momentum_boundarylayer(transition[0], u, g)
            warningtext.set_visible(False)
            blline.set_xdata(x)
            blline.set_ydata(bl*1000)
            trline.set_xdata(transition[0])
            trline.set_ydata(transition_bl*1000)
            fig.gca().set_xlim([0, platelength])
            fig.gca().set_ylim([0, np.max(bl)*1000])
        else:
            warningtext.set_visible(True)
            blline.set_xdata(0)
            blline.set_ydata(0)
            trline.set_xdata(0)
            trline.set_ydata(0)
        fig.canvas.draw()
        
    #def resize(event):
    #    print('resize')
    #    dpi = fig.get_dpi()
    #    #fig.set_size_inches(event.width/dpi, event.height/dpi)
    #    warningtext.set_visible(True)
    #    warningtext.set_text(event.width/dpi)
        
    #fig.canvas.mpl_connect('resize_event', resize)
            
    return update
        

In [22]:
gas_dropdown = widgets.Dropdown(options=gas.state.fluids_list(), value='Air')

pwidget, pslider, _ = quantity('Pressure in Pa:', min=1, max=150e5, step=1e5, value=101325)
Twidget, Tslider, _ = quantity('Temperature in °C:', min=-30, max=55, step=1, value=20)

thermostate = widgets.VBox([
    pwidget,
    Twidget,
    widgets.HBox([widgets.Label('Gas / Specie Mixture:'), gas_dropdown])
])

platewidget, plateslider, _ = quantity('Platelength in m', min=0.001, max=10, step=0.001, value=0.1)
uwidget, uslider, _ = quantity('Velocity in m/s:', min=0.001, max=200, step=0.001, value=20)

flow = widgets.VBox([platewidget, uwidget])

accordion = widgets.Accordion(children=[thermostate, flow])
accordion.set_title(0, 'Thermodynamic State')
accordion.set_title(1, 'Flat Plate')

def interactive_output(f, controls, layout=None):
    """Connect widget controls to a function.
    This function does not generate a user interface for the widgets (unlike `interact`).
    This enables customisation of the widget user interface layout.
    The user interface layout must be defined and displayed manually.
    """
    from IPython.display import clear_output
    out = widgets.Output(layout=layout)
    def observer(change):
        kwargs = {k:v.value for k,v in controls.items()}
        #widgets.show_inline_matplotlib_plots()
        with out:
            
            clear_output(wait=True)
            f(**kwargs)
            #widgets.show_inline_matplotlib_plots()
    for k,w in controls.items():
        w.observe(observer, 'value')
    #widgets.show_inline_matplotlib_plots()
    observer(None)
    return out

update_plot = plot_init()
#interactive_plot = widget.Output(widgets.Layout(display='flex', width='100%'))

interactive_plot = interactive_output(update_plot,
                                      {'platelength': plateslider,
                                       'u': uslider,
                                       'pressure': pslider,
                                       'temperature': Tslider,
                                       'gasname': gas_dropdown},
                                     layout=widgets.Layout(display='flex', width='100%'))
#interactive_plot.layout = widgets.Layout(display='flex', width='100%')

display(accordion, interactive_plot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Accordion(children=(VBox(children=(HBox(children=(Label(value='Pressure in Pa:', layout=Layout(flex='1 1 auto'…

Output(layout=Layout(display='flex', width='100%'))

In [25]:
%run '../_libtoolbox/footer.py'


<div style='position: fixed; left: 0; bottom: 0; width: 100%; background-color: #f7b500; color: #000000; text-align: center; vertical-align: middle; padding: 10px'> 
<b><a style='color: #000000;' href="https://nuflo.de/"><img src="../_libtoolbox/nuflo-logo-N-512x512.png" alt="logo-N" width="16px" />&nbsp;&nbsp;&nbsp;nuflo</a></b></div>